# Using beautifulsoup to scrape the 104 data analyst job in Taipei and New Taipei city
with title, salary, company, location, sector, experience_req, edu_req, description

In [158]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

job_title = []
salary = []
company = []
sector = []
experience_req = []
edu_req = []
job_description = []
location = []



for page in range(1, 11):
    # Setting url changing with dif pages
    url = f'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=data%20analyst&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page={page}&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        # Using find_all with setting tag and attribute to find titles 
        title_list = soup.find_all('a', class_='js-job-link', href=re.compile("relevance"))

        # Extract job title and link and append to list
        for t in title_list:
            title = t.get_text().strip()
            job_title.append(title)
            job_lin = t['href']

            # Extract description and salary in job link    
            job_link = f'https:{job_lin}'
            job_response = requests.get(job_link)
            if job_response.status_code == 200:
                job_soup = BeautifulSoup(job_response.text, 'html.parser')
                desc = job_soup.find('p', class_='mb-5 r3 job-description__content text-break')
                # prevent description can't be extract
                if desc:
                    description = desc.get_text().strip()
                    job_description.append(description)
                else:
                    job_description.append("Description not available")  # Handle the case when description is not found
        
                sal_list = job_soup.find('p', class_='t3 mb-0 mr-2 text-primary font-weight-bold align-top d-inline-block')
                sal = sal_list.text.strip()
                salary.append(sal)

        company_sector_list = soup.find_all('article', {'data-jobsource': re.compile("relevance")})
        for c in company_sector_list:
            com = c.get('data-cust-name')
            sec = c.get('data-indcat-desc')
            company.append(com)
            sector.append(sec)
        
        requirement_list = soup.find_all('article', {'data-jobsource': re.compile("relevance")})
        requirement = []
        for r in requirement_list:
            req = r.find('ul', class_="b-list-inline b-clearfix job-list-intro b-content").get_text()
            requirement.append(req)
            
        for l in requirement:
            loc = l.split('\n')[1]
            location.append(loc) 

        for e in requirement:
            exp = e.split('\n')[3]
            experience_req.append(exp)
            
        for e in requirement:
            edu = e.split('\n')[4]
            edu_req.append(edu)

In [160]:
data = {'title':job_title, 'salary':salary, 'company':company, 'location':location, 'sector':sector, 'experience_req':experience_req, 'education_req':edu_req, 'description':job_description}
df = pd.DataFrame(data)

In [177]:
df

,title,salary,company,location,sector,experience_req,education_req,description
0,資深數據分析師 Senior Data Analyst,待遇面議,Gogolook_走著瞧股份有限公司,台北市中正區,電腦軟體服務業,5年以上,大學,Job description\nGogolook has been deeply invo...
1,雲端數據分析師 Data Analyst／Data Scientist,"月薪35,000~50,000元",瀚荃股份有限公司,新北市淡水區,其他電子零組件相關業,1年以上,大學,1. 具有統計概念，並擁有好的邏輯思維與簡報技巧\n2. 擁有 BI 工具使用經驗（如：Po...
2,行銷數據分析師 Marketing Data Analyst,待遇面議,以沛有限公司,台北市松山區,廣告行銷公關業,3年以上,大學,◎ 關於你——行銷數據分析師\nData Team 可說是 uppeta 最堅實的營運基礎，...
3,After-serivce Data Analyst 售後服務規劃數據分析師,待遇面議,Gogoro Taiwan Limited_睿能創意股份有限公司,台北市松山區,機車及其零件製造業,2年以上,大學,【職務影響力 Position Impact】\nAfter-serivce Data An...
4,【採線上面談】Data Analyst 資料分析師,"月薪50,000~90,000元",新加坡商鈦坦科技股份有限公司台灣分公司,台北市南港區,電腦軟體服務業,經歷不拘,學歷不拘,"【Game changer, armed with data.】\nYou know all..."
...,...,...,...,...,...,...,...,...
195,商業分析師 Business analyst,待遇面議,賽普勒斯商飛力通股份有限公司台灣分公司,台北市內湖區,電腦軟體服務業,2年以上,大學,(一)工作內容：\n1.市場情報蒐集及調查分析\n2.專案產品設定與客戶操作流程分析\n3....
196,數據分析師_0304,待遇面議,遊戲橘子數位科技股份有限公司,台北市內湖區,其它軟體及網路相關業,1年以上,大學,1.運用分析工具(如：:Tableau)，進行資料視覺化圖表製作\n2.從營運數據中進行趨勢...
197,ECN Analyst (Engineer Change Notice),待遇面議,美商ID TECH台灣分公司_璟昇科技股份有限公司,桃園市桃園區,電子通訊／電腦週邊零售業,3年以上,大學,• Engineering Change Notice(ECN)Analyst suppor...
198,Taiwan Office_Business Intelligence Analyst,待遇面議,Gogoro Taiwan Limited_睿能創意股份有限公司,台北市松山區,機車及其零件製造業,3年以上,大學,Responsibilities:\n• Build business intelligen...


In [176]:
df.to_csv('104 data analyst job scrape.csv', encoding="utf_8_sig")